# Monkey patching in fastai
> A case study with `Path.ls`

- toc: true 
- badges: true
- comments: true
- categories: [programming]

# Monkey patching
>"We do it in fastai all the time."
>   
>Jeremy Howard

from [Swift For TensorFlow AND FastAI: Part 2](https://www.youtube.com/watch?v=Zr4QI64kUI4&list=PLderfcX9H9Mq8hv4W2ixqSTynwvBn9ppD&t=9m12s) (featuring Chris Lattner).

Monkey patching refers to the practise of dynamically modifying classes and modules.  This is done quite frequently in the `fastai` library.

This notebook dissects one common example, namely the [`ls()` method](https://fastcore.fast.ai/utils#Path.ls) added to the `Path` class from the `pathlib` `Python` library, which functionalities reminiscent of the [`ls` command in `bash`](https://ss64.com/bash/ls.html).  From the `fastcore` documentation:  
>We add an `ls()` method to `pathlib.Path` which is simply defined as `list(Path.iterdir())`, mainly for convenience in REPL environments such as notebooks.

---
tags: programming python fastai exploration

# The `pathlib` module

The [`pathlib`](https://docs.python.org/3/library/pathlib.html) module provides functionalities to handle filesystem paths.

In [1]:
from pathlib import Path

path = Path().home()
dir_native = dir(path)
print(f"Path to home directory:         {str(path)}")
print(f'Object path has attribute ls:   {"ls" in dir_native}')

Path to home directory:         /Users/antoine
Object path has attribute ls:   False


# Supercharging `Path` objects with `fastai`

We first remove the `Path` class from our scope so that we can load it again via `fastai` (read on):

In [7]:
%xdel Path
try:
    Path
    print("This will fail so this won't print")
except NameError:
    print("Class `Path` has been removed.")

NameError: name 'Path' is not defined
Class `Path` has been removed.


The following will automatically load `Path` (among many other things):

In [3]:
from fastai2.vision.all import *

path = Path().home()
dir_fastai = dir(path)
print(f"Path to home directory:         {str(path)}")
print(f'Object path has attribute ls:   {"ls" in dir_fastai}')

Path to home directory:         /Users/antoine
Object path has attribute ls:   True


The `ls` method is one of several attributes that `fastai` (in fact, `fastcore`, read on) adds to the `Path` class.  The list of added attributes is:

In [4]:
set(dir_fastai) - set(dir_native)

{'load',
 'load_array',
 'ls',
 'read',
 'readlines',
 'save',
 'save_array',
 'write'}

# Loading `Path` and `Path.ls` only?

It is an instructive exercise to find out how to load `Path` and its attributes added in the `fastai` library only.  This is done in another notebook.